#### 1244. Design A Leaderboard

* https://algo.monster/liteproblems/1244

#### BBG - IMP

In [ ]:
# Time complexity of below solution using SortedDict is better but it uses SortedDict
# If we need a solution using standard python lib than we can use heap with lazy deletion
# Heavy lifting in this case would be done in topk method

import heapq
from typing import List, Dict, Tuple

class Leaderboard:
    """
    | Operation  | Time           | Reason           |
    | ---------- | -------------- | ---------------- |
    | `addScore` | **O(log N)**   | Heap push        |
    | `top(K)`   | **O(K log N)** | Heap pop K times |
    | `reset`    | **O(1)**       | Dict update      |
    | Space      | **O(N)**       | Heap + dict      |
    """
    def __init__(self):
        self.player_scores: Dict[int, int] = {} # player_id -> score
        self.max_heap: List[int] = [] # (score, player_id)

    def add_score(self, player_id: int, score: int) -> None:
        new_score: int = self.player_scores.get(player_id, 0) + score
        self.player_scores[player_id] = new_score
        heapq.heappush(self.max_heap, (-new_score, player_id))

    def top(self, k: int) -> int:
        total_score: int = 0
        popped_elements: List[Tuple[int, int]] = []
        
        while k > 0 and self.max_heap:
            neg_score, player_id = heapq.heappop(self.max_heap)
            current_score = self.player_scores[player_id]

            # Lazy deletion
            if -neg_score != current_score:
                continue

            total_score += current_score
            k -= 1

            popped_elements.append((neg_score, player_id))
        
        # push back valid entries to preserve heap state
        for neg_score, player_id in popped_elements:
            heapq.heappush(self.max_heap, (neg_score, player_id))
        
        return total_score


    def reset(self, player_id: int) -> None:
        self.player_scores[player_id] = 0

lb = Leaderboard()
lb.add_score('p1', 100)
lb.add_score('p2', 200)
lb.add_score('p3', 100)
lb.add_score('p4', 400)
print(lb.player_scores)
print(lb.max_heap)
print(lb.top(2))
lb.add_score('p1', 600)
print(lb.player_scores)
print(lb.max_heap)
# lb.reset('p3')
print(lb.player_scores)
print(lb.max_heap)
print(lb.top(4))
print(lb.player_scores)
print(lb.max_heap)

{'p1': 100, 'p2': 200, 'p3': 100, 'p4': 400}
[(-400, 'p4'), (-200, 'p2'), (-100, 'p3'), (-100, 'p1')]
600
{'p1': 700, 'p2': 200, 'p3': 100, 'p4': 400}
[(-700, 'p1'), (-400, 'p4'), (-100, 'p1'), (-100, 'p3'), (-200, 'p2')]
{'p1': 700, 'p2': 200, 'p3': 100, 'p4': 400}
[(-700, 'p1'), (-400, 'p4'), (-100, 'p1'), (-100, 'p3'), (-200, 'p2')]
1400
{'p1': 700, 'p2': 200, 'p3': 100, 'p4': 400}
[(-700, 'p1'), (-400, 'p4'), (-200, 'p2'), (-100, 'p3')]


In [ ]:
from sortedcontainers import SortedDict
from typing import List, Dict, Set

class Leaderboard:
    """
        ------------------------------------------------------------------
        | Operation  | Time Complexity                | Space Complexity |
        | ---------- | ------------------------------ | ---------------- |
        | `addScore` | O(log N) – update `SortedDict` | O(N)             |
        | `top(K)`   | O(K)                           | O(1)             |
        | `reset`    | O(log N)                       | O(N)             |
        ------------------------------------------------------------------
    """
    def __init__(self):
        self.player_scores: Dict[int, int] = {} # map playerid -> score
        self.score_board: Dict[int, Set[int]] = SortedDict() # score -> set(player_ids)

    def add_score(self, player_id: int, score: int) -> None:
        """Add score if the player is new else update score"""
        old_score: int = self.player_scores.get(player_id, 0)
        new_score: int = old_score + score
        self.player_scores[player_id] = new_score

        if old_score > 0:
            players = self.score_board[old_score]
            players.remove(player_id)
            if not players:
                del self.score_board[old_score]
            
        if not new_score in self.score_board:
            self.score_board[new_score] = set()
        self.score_board[new_score].add(player_id)

    def top(self, k: int) -> int:
        """ Return the total of top k players"""
        total: int = 0
        count: int = 0

        for score in reversed(self.score_board):
            for _ in self.score_board[score]:
                total += score
                count += 1
                if count == k:
                    return total
        return total


    def reset(self, player_id: int) -> None:
        if player_id in self.player_scores:
            score = self.player_scores[player_id]
            players_list = self.score_board[score]
            players_list.remove(player_id)
            if not players_list:
                del self.score_board[score]
        self.player_scores[player_id] = 0


lb = Leaderboard()
lb.add_score('p1', 100)
lb.add_score('p2', 200)
lb.add_score('p3', 100)
lb.add_score('p4', 400)
print(lb.player_scores)
print(lb.score_board)
print(lb.top(2))
lb.add_score('p1', 600)
print(lb.player_scores)
print(lb.score_board)
lb.reset('p3')
print(lb.player_scores)
print(lb.score_board)

{'p1': 100, 'p2': 200, 'p3': 100, 'p4': 400}
SortedDict({100: {'p1', 'p3'}, 200: {'p2'}, 400: {'p4'}})
600
{'p1': 700, 'p2': 200, 'p3': 100, 'p4': 400}
SortedDict({100: {'p3'}, 200: {'p2'}, 400: {'p4'}, 700: {'p1'}})
{'p1': 700, 'p2': 200, 'p3': 0, 'p4': 400}
SortedDict({200: {'p2'}, 400: {'p4'}, 700: {'p1'}})


In [7]:
import heapq
class StockTick:
    def __init__(self):
        self.stock_vol = {}
        self.vol_stock_max_heap = []

    def add_stocks_volume(self, stock, volume):
        new_vol = self.stock_vol.get(stock, 0) + volume
        self.stock_vol[stock] = new_vol

        heapq.heappush(self.vol_stock_max_heap, (-new_vol, stock))

    def top_k_stocks(self, k):
        stock_list = []
        popped_elements = []

        while k and self.vol_stock_max_heap:
            neg_vol, stock = heapq.heappop(self.vol_stock_max_heap)

            curr_vol = self.stock_vol[stock]

            if -neg_vol != curr_vol:
                continue

            stock_list.append((stock, curr_vol))
            k -= 1

            popped_elements.append((neg_vol, stock))
        
        for neg_vol, stock in popped_elements:
            heapq.heappush(self.vol_stock_max_heap, (neg_vol, stock))

        return stock_list
    

st = StockTick()
st.add_stocks_volume('sbi', 800)
st.add_stocks_volume('itc', 200)
st.add_stocks_volume('kotak', 1000)
st.add_stocks_volume('itc', 400)
st.add_stocks_volume('sbi', 300)

st.top_k_stocks(2)
st.add_stocks_volume('itc', 1400)
st.add_stocks_volume('sbi', 400)
st.top_k_stocks(2)

[('itc', 2000), ('sbi', 1500)]

In [ ]:
import heapq
from sortedcontainers import SortedDict
class StockTick:
    def __init__(self):
        self.stock_vol = {}
        self.vol_stock_dict = SortedDict()

    def add_stocks_volume(self, stock, volume):
        old_vol = self.vol_stock_dict.get(stock, 0)
        new_vol = volume + old_vol
        self.stock_vol[stock] = new_vol

        if old_vol > 0:
            stock_list = self.vol_stock_dict[old_vol]
            stock_list.remove(stock)
            

    def top_k_stocks(self, k):
        stock_list = []
        popped_elements = []

        while k and self.vol_stock_max_heap:
            neg_vol, stock = heapq.heappop(self.vol_stock_max_heap)

            curr_vol = self.stock_vol[stock]

            if -neg_vol != curr_vol:
                continue

            stock_list.append((stock, curr_vol))
            k -= 1

            popped_elements.append((neg_vol, stock))
        
        for neg_vol, stock in popped_elements:
            heapq.heappush(self.vol_stock_max_heap, (neg_vol, stock))

        return stock_list
    

st = StockTick()
st.add_stocks_volume('sbi', 800)
st.add_stocks_volume('itc', 200)
st.add_stocks_volume('kotak', 1000)
st.add_stocks_volume('itc', 400)
st.add_stocks_volume('sbi', 300)

st.top_k_stocks(2)
st.add_stocks_volume('itc', 1400)
st.add_stocks_volume('sbi', 400)
st.top_k_stocks(2)